# Tempo Client Demo

In [1]:
import sys
import asyncio
import aiohttp
import os
import pandas as pd
from typing import List, Dict

sys.path.append ('..')
import utils

project_path = os.path.join('..', '..', 'scale')

sys.path.append(project_path)

utils.add_to_sys_path(project_path)


from scale.backends.tempo_client import TempoClient
from scale.modeler.src.models import Span, Trace

## Forward the tempo port to localhost

If you cannot reach the node port, or just want to forward it anyway

**NOTE:** run this from another command window

In [ ]:
%%bash

kubectl port-forward -n monitoring svc/tempo 3200:3200 

In [7]:
TEMPO_URL = "http://localhost:3200"

# If using nodeport
# TEMPO_URL = "http://192.168.49.2:32000"

## Create TempoClient instance

In [39]:

tempo_client = TempoClient(TEMPO_URL)

## Check the TempoClient API

In [40]:
help(TempoClient)

Help on class TempoClient in module scale.backends.tempo_client:

class TempoClient(builtins.object)
 |  TempoClient(tempo_url)
 |
 |  Convenience class for interacting with Tempo gRPC API
 |
 |  Methods defined here:
 |
 |  __init__(self, tempo_url)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  async build_span_df(self, start_delta: int, limit: int) -> pandas.core.frame.DataFrame
 |      Generate a pandas data frame containing trace data in the following form
 |      [TraceID,SpanID,ParentID,OperationName,StartTimeUnixNano,Duration]
 |
 |      Args:
 |          start_delta (int): Number of seconds in past to search for traces.
 |          limit (int): Limit number of records from Tempo. Defaults to 1000.
 |
 |      Returns:
 |          pd.DataFrame: data frame containing trace data
 |
 |  async find_trace_by_id(self, session: aiohttp.client.ClientSession, trace_id: str) -> opentelemetry.proto.trace.v1.trace_pb2.TracesData
 |      Find a trace in Tempo b

## Get Recent Trace IDs

In [41]:
tcp_connection = aiohttp.TCPConnector(limit=20)
start_delta = 60 # get the last minute of traces

async with aiohttp.ClientSession(connector=tcp_connection) as session:
    trace_ids = await tempo_client.search(session=session, start_delta=start_delta, limit=10)

trace_ids
    

['6411b1dfe627b258d4a0ee1c1e6c0b9',
 '47ed37eb8979935be8d21a90d29fade',
 'abf78b84f9b3d35a47bcb32a2cfed83',
 '884997d3f6d6bafbf275f174f622524',
 '6882ac8b7ac8a901c3fee34f81a365',
 'c041c1297a474e53a971e798194ad0f',
 '8a8a3d5e8f24ff969e6d6505b2e76a3',
 'a48b537294d6770be240d5121859ccd',
 '968df5850ffa32b525c263bac78dd54',
 '12661c50006cff76394966c5252100']

## Get Trace By ID

In [42]:
tcp_connection = aiohttp.TCPConnector(limit=20)
start_delta = 60 # get the last minute of traces

trace_id = trace_ids[1]
async with aiohttp.ClientSession(connector=tcp_connection) as session:
    trace = await tempo_client.find_trace_by_id(session=session, trace_id=trace_id)

trace

resource_spans {
  resource {
    attributes {
      key: "telemetry.sdk.language"
      value {
        string_value: "python"
      }
    }
    attributes {
      key: "telemetry.sdk.name"
      value {
        string_value: "opentelemetry"
      }
    }
    attributes {
      key: "telemetry.sdk.version"
      value {
        string_value: "1.23.0"
      }
    }
    attributes {
      key: "service.namespace"
      value {
        string_value: "opentelemetry-demo"
      }
    }
    attributes {
      key: "service.version"
      value {
        string_value: "1.11.1"
      }
    }
    attributes {
      key: "service.name"
      value {
        string_value: "loadgenerator"
      }
    }
  }
  scope_spans {
    scope {
      name: "opentelemetry.instrumentation.requests"
      version: "0.44b0"
    }
    spans {
      trace_id: "\004~\323~\270\227\2315\276\215!\251\r)\372\336"
      span_id: "h\253\300U\337\370\030\302"
      name: "GET"
      kind: SPAN_KIND_CLIENT
      start_tim

## Model the trace

In [43]:
from google.protobuf.json_format import MessageToDict

attr_keys = ('string_value', 'int_value', 'double_value', 'bool_value')

spans = []

for resource_span in trace.resource_spans:
    # comprehension to extract resource attributes
    # transform attribute keys to snake_case where attribute key is a dot-separated string
    resource_attributes = {attr.key.replace('.', '_'): getattr(attr.value, key) 
                            for attr in resource_span.resource.attributes 
                            for key in attr_keys 
                            if attr.value.HasField(key)}


    for scope_span in resource_span.scope_spans:
        scope = scope_span.scope
        scope_name = scope.name
        scope_version = scope.version

        for span in scope_span.spans:

            # Convert span protobuf message to a dictionary
            span_dict = MessageToDict(span, preserving_proto_field_name=True)
            
            # Add resource-level attributes to the span data (e.g., service.name)
            span_dict.update(resource_attributes)
            
            # Add scope name and version
            span_dict['scope_name'] = scope_name
            span_dict['scope_version'] = scope_version

            spans.append(Span.from_dict(span_dict))


trace = Trace(trace_id, spans)

trace.to_df()

,trace_id,span_id,name,kind,start_time_unix_nano,end_time_unix_nano,attributes,status,telemetry_sdk_language,telemetry_sdk_name,...,process_executable_name,process_executable_path,process_runtime_version,process_runtime_name,process_runtime_description,process_command,process_owner,telemetry_auto_version,events,os_description
0,BH7TfriXmTW+jSGpDSn63g==,aKvAVd/4GMI=,GET,SPAN_KIND_CLIENT,1730002534903897570,1730002534911258581,"[{'key': 'http.method', 'value': {'string_valu...",{},python,opentelemetry,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BH7TfriXmTW+jSGpDSn63g==,gFF/TVfW/Oc=,GET,SPAN_KIND_SERVER,1730002534904000000,1730002534909618453,"[{'key': 'http.host', 'value': {'string_value'...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
2,BH7TfriXmTW+jSGpDSn63g==,zSHmCbKi2Vw=,ingress,SPAN_KIND_SERVER,1730002534904568000,1730002534910532000,"[{'key': 'node_id', 'value': {'string_value': ...",{},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BH7TfriXmTW+jSGpDSn63g==,FkjlpwGzvEc=,router frontend egress,SPAN_KIND_CLIENT,1730002534904662000,1730002534910505000,"[{'key': 'http.protocol', 'value': {'string_va...",{},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BH7TfriXmTW+jSGpDSn63g==,MOQVzoPC58k=,GET /api/recommendations?productIds=66VCHSJNUP,SPAN_KIND_SERVER,1730002534905000000,1730002534909727600,"[{'key': 'next.span_name', 'value': {'string_v...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
5,BH7TfriXmTW+jSGpDSn63g==,5lraquDgfQo=,executing api route (pages) /api/recommendations,SPAN_KIND_INTERNAL,1730002534905000000,1730002534909474036,"[{'key': 'next.span_name', 'value': {'string_v...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
6,BH7TfriXmTW+jSGpDSn63g==,2epZvrRX/Ro=,grpc.oteldemo.RecommendationService/ListRecomm...,SPAN_KIND_CLIENT,1730002534905000000,1730002534907906271,"[{'key': 'rpc.system', 'value': {'string_value...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
7,BH7TfriXmTW+jSGpDSn63g==,Opzo/BAQJmQ=,grpc.oteldemo.ProductCatalogService/GetProduct,SPAN_KIND_CLIENT,1730002534908000000,1730002534908942814,"[{'key': 'rpc.system', 'value': {'string_value...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
8,BH7TfriXmTW+jSGpDSn63g==,SFk8xYs/w6Y=,grpc.oteldemo.ProductCatalogService/GetProduct,SPAN_KIND_CLIENT,1730002534908000000,1730002534909073175,"[{'key': 'rpc.system', 'value': {'string_value...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
9,BH7TfriXmTW+jSGpDSn63g==,XAYAYxjNT+E=,grpc.oteldemo.ProductCatalogService/GetProduct,SPAN_KIND_CLIENT,1730002534908000000,1730002534908942665,"[{'key': 'rpc.system', 'value': {'string_value...",{},nodejs,opentelemetry,...,node,/usr/local/bin/node,20.16.0,nodejs,Node.js,/app/server.js,nextjs,NaN,NaN,NaN
